<a href="https://colab.research.google.com/github/vanshitabansal/Bounding_box_generation/blob/main/bounding_box_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import pandas as pd
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import cv2
from glob import glob
from tqdm import tqdm 
import csv
from pathlib import Path
from skimage.measure import label, regionprops, find_contours

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Get all the images (.png) using glob
benign_images=glob(os.path.join("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/dataset/benign","*.png"))
malignant_images = glob(os.path.join("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/dataset/malignant","*.png"))
normal_images = glob(os.path.join("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/dataset/normal","*.png"))

# check if the path exist or not, if not create it to save the output
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

create_dir("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Modified/benign")
create_dir("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Modified/malignant")
create_dir("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Modified/normal")

# Loop over the dataset to resize the images if required
for x in benign_images: 
    name = x.split("/")[-1].split(".")[0]
    image = Image.open(x)
    img=cv2.imread(x)
    img = cv2.resize(img, (256,256))
    im = Image.fromarray(img)
    im.save(f"/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Modified/benign/{name}.png")

for x in malignant_images:
    name = x.split("/")[-1].split(".")[0]
    image = Image.open(x)
    img=cv2.imread(x)
    img = cv2.resize(img, (256,256))
    im = Image.fromarray(img)
    im.save(f"/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Modified/malignant/{name}.png")

for x in normal_images:
    name = x.split("/")[-1].split(".")[0]
    image = Image.open(x)
    img=cv2.imread(x)
    img = cv2.resize(img, (256,256))
    im = Image.fromarray(img)
    im.save(f"/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Modified/normal/{name}.png")

In [ ]:
#  To get only boundaries from the mask
def mask_to_border(mask):
    h , w = mask.shape

    #  creating blank image of same size
    border = np.zeros ((h , w))

    contours = find_contours(mask, 128)
    for contour in contours:
        for c in contour:
            x = int(c[0])
            y = int(c[1])
            border[x][y] = 255 
    
    return border

def mask_to_bbox(mask):
    bboxes = []

    border = mask_to_border(mask)
    lbl = label(border)
    props = regionprops(lbl)

    for prop in props:
        x1 = prop.bbox[1]
        y1 = prop.bbox[0]

        x2 = prop.bbox[3]
        y2 = prop.bbox[2]

        bboxes.append([x1,y1,x2,y2])

    return bboxes

# field names for csv file
fields = ['image_id', 'Width', 'Height', 'bbox'] 

# name of csv file 
filename = "/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/train/train.csv"
    


# main function
if __name__ == "__main__":
    # load the dataset
    benign_images =glob(os.path.join("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Modified/benign","*).png"))
    benign_mask = glob(os.path.join("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Modified/benign","*mask*.png"))
    malignant_images = glob(os.path.join("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Modified/malignant","*).png"))
    malignant_mask = glob(os.path.join("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Modified/malignant","*mask*.png"))
    normal_images = glob(os.path.join("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Modified/normal","*).png"))
    normal_mask = glob(os.path.join("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Modified/normal","*mask*.png"))

    
    # Create a folder to save images
    create_dir("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Results/benign")
    create_dir("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Results/malignant")
    create_dir("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Results/normal")

    # Loop over the benign dataset
    rows=[[]]
    for x in benign_images:
        name = x.split("/")[-1].split(".")[0]
        bboxes = []
        flag = 0

        # Read the image
        xx = cv2.imread(x,cv2.IMREAD_COLOR)
        for y in benign_mask:
             name2 = y.split("/")[-1].split(".")[0]
             if(name2.startswith(name)):

                # Read the mask 
                yy = cv2.imread(y,cv2.IMREAD_GRAYSCALE)
                
                # Detecting bounding boxes
                bboxes=mask_to_bbox(yy)

                # Marking bounding boxes in image
                coordinates=[]
                for bbox in bboxes:
                    xx = cv2.rectangle(xx,(bbox[0],bbox[1]),(bbox[2],bbox[3]),(255,0,0),2)
                    coordinates.extend([bbox[0],bbox[1],bbox[2],bbox[3]])
                    rows.append([name,256,256,coordinates])
                    # Save the images
                    cv2.imwrite(f"/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Results/benign/{name}.png",xx)
                # print(coordinates)
                
    # print(rows)
    # print(len(rows))
    # Loop over the malignant dataset
    for x in malignant_images:
        name = x.split("/")[-1].split(".")[0]
        bboxes = []
        flag = 0

        # Read the image
        xx = cv2.imread(x,cv2.IMREAD_COLOR)
        for y in malignant_mask:
             name2 = y.split("/")[-1].split(".")[0]
             if(name2.startswith(name)):

                # Read the mask 
                yy = cv2.imread(y,cv2.IMREAD_GRAYSCALE)
                
                # Detecting bounding boxes
                bboxes=mask_to_bbox(yy)

                # Marking bounding boxes in image
                coordinates=[]
                for bbox in bboxes:
                    xx = cv2.rectangle(xx,(bbox[0],bbox[1]),(bbox[2],bbox[3]),(255,0,0),2)
                    coordinates.extend([bbox[0],bbox[1],bbox[2],bbox[3]])
                    rows.append([name,256,256,coordinates])
                    # Save the images
                    cv2.imwrite(f"/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Results/malignant/{name}.png",xx)

    
   
    
    for x,y in tqdm(zip(normal_images,normal_mask),total=len(normal_mask)):

        # Extracting the name of image and also removing .png
        name = y.split("/")[-1].split(".")[0]
        # Read the image and mask
        x = cv2.imread(x,cv2.IMREAD_COLOR)
        y= cv2.imread(y,cv2.IMREAD_GRAYSCALE)

        # Detecting bounding boxes 
        bboxes = mask_to_bbox(y)

        # Marking bounding boxes in image
        coordinates=[]
        for bbox in bboxes:
            x = cv2.rectangle(x,(bbox[0],bbox[1]),(bbox[2],bbox[3]),(255,0,0),2)
            coordinates.extend([bbox[0],bbox[1],bbox[2],bbox[3]])
            rows.append([name,256,256,coordinates])
        # Save the images
        cv2.imwrite(f"/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Results/normal/{name}.png",x)

print(rows)
# writing to csv file 
with open(filename, 'w',newline='') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(fields) 
        
    # writing the data rows 
    csvwriter.writerows(rows)

